In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict, ChainMap
import glob
import sys 
import functools
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import dask.dataframe as dd
from multiprocessing import Pool, Manager, cpu_count
from joblib import Parallel, delayed
import pymp
import gezi
# from gezi import tqdm
from tqdm import tqdm
tqdm.pandas()

In [2]:
timer = gezi.Timer('read user_action2.feather', True)
d = pd.read_feather('../input/user_action2.feather')
d.feedid = d.feedid.astype(int)
d.userid = d.userid.astype(int)
d.date_ = d.date_.astype(int)
timer.print()

read user_action2.feather start
read user_action2.feather duration: 1.5295531749725342


In [3]:
# d.head()

In [8]:
# d[(d.date_ == 14) & (d.actions == 0)]

In [9]:
ACTIONS = [
  'read_comment',
  'comment',
  'like',
  'click_avatar',
  'forward',
  'follow',
  'favorite'
]

ACTIONS2 = ACTIONS + ['finish', 'stay']

HIS_ACTIONS = ACTIONS2 + [
    'pos', 'neg', 'unfinish', 'unstay', \
    #     'unfinish2', 'unstay2', 
#     'show', 
#   'pos_action', 'neg_action', 'show_action'
]

In [10]:
def is_neg(row):
  return row['actions'] == 0

d = d.sort_values(['date_'], ascending=False)
# TODO 如果能看到test集合 那么需要先把test集合的shows加上

userids = [int(x.strip().split()[0]) for x in open('../input/user_vocab.txt').readlines()]
print('num_userids', len(userids))
history = {}
for userid in userids:
  history[userid] = {}
  for action in HIS_ACTIONS:
    history[userid][action] = []

def gen_history_row(row, history):
  feedid = int(row['feedid'])
  userid = int(row['userid'])
  day = int(row['date_'])
  feedid = (feedid, day)
  his = history[userid]
#   his['show'].append(feedid)
  if row['finish_rate'] > 0.99:
    his['finish'].append(feedid)
  if row['finish_rate'] < 0.01:
    his['unfinish'].append(feedid)
#   if row['finish_rate'] < 0.1:
#     his['unfinish2'].append(feedid)
  if row['stay_rate'] > 1:
    his['stay'].append(feedid)
  if row['stay_rate'] < 0.01:
    his['unstay'].append(feedid)
#   if row['stay_rate'] < 0.1:
#     his['unstay2'].append(feedid)
  actions = [0] * len(ACTIONS2)
  for i, action in enumerate(ACTIONS):
    if row[action] > 0:
      his[action].append(feedid)
      actions[i] = 1
  actions[-2] = row['finish_rate']
  actions[-1] = row['stay_rate']
  is_neg_row = is_neg(row)
  if not is_neg_row:
    his['pos'].append(feedid)
#     his['pos_action'].append(actions)
  else:
    his['neg'].append(feedid)

#   his['show_action'].append(actions)

def gen_history(d, userids=None):
  history = {}
  if userids is not None:
    d = d[d.userid.isin(set(userids))]
  else:
    userids = set(d.userid)
  for userid in userids:
    history[userid] = {}
    for action in HIS_ACTIONS:
      history[userid][action] = []

  for row in tqdm(d.itertuples(), total=len(d), desc=f'user_action users:{len(userids)}'):
    row = row._asdict()
    gen_history_row(row, history)
  return history  
    
# nw = 12
# res = Parallel(n_jobs=nw)(delayed(lambda x: gen_history(d, x))(uids) for uids in np.array_split(userids, nw))
# print(res)
# d.progress_apply(gen_history_row, axis=1)
# d.parallel_apply(gen_history_row, axis=1)

## 可能需要meta 较为麻烦
# d2 = dd.from_pandas(d,npartitions=40)

nw = cpu_count()
userids_list = np.array_split(userids, nw)
res = Manager().dict()
with pymp.Parallel(nw) as p:
  for i in p.range(nw):
    res[i] = gen_history(d, userids_list[i])
with gezi.Timer('ChainMap merge res', True):
  history = dict(ChainMap(*res.values()))

# nw = cpu_count()
# userids_list = np.array_split(userids, nw)
# pfunc = functools.partial(gen_history, d=d)
# with Pool(nw) as p:
#   res = p.map(pfunc, userids_list)
# with timer = gezi.Timer('ChainMap merge res', True):
#   history = dict(ChainMap(*res))


num_userids 20000


user_action users:208: 100%|██████████| 34415/34415 [00:00<00:00, 92906.14it/s]


user_action users:208: 100%|██████████| 53382/53382 [00:00<00:00, 94344.97it/s]

user_action users:208:  83%|████████▎ | 21383/25624 [00:00<00:00, 107011.78it/s]

user_action users:209: 100%|██████████| 103008/103008 [00:01<00:00, 97545.50it/s]

user_action users:208: 100%|██████████| 25624/25624 [00:00<00:00, 107353.32it/s]

user_action users:208: 100%|██████████| 14673/14673 [00:00<00:00, 102410.07it/s]


user_action users:209: 100%|██████████| 145560/145560 [00:01<00:00, 88034.83it/s] 

user_action users:209: 100%|██████████| 182240/182240 [00:01<00:00, 94752.67it/s] 


ChainMap merge res start
ChainMap merge res duration: 5.080284118652344


In [11]:
len(history)

20000

In [12]:
# history[userids[0]]

In [13]:
with gezi.Timer('save history.pkl', True):
  gezi.save_pickle(history, '../input/history.pkl')

save history.pkl start
save history.pkl duration: 3.2910330295562744


In [14]:
m = {
  'userid': [],
  'pos': [],
  'pos_len': [],
  'finish': [],
  'finish_len': [],
  'stay': [],
  'stay_len': [],
  'neg': [],
  'neg_len': [],
  'unfinish': [],
  'unfinish_len': [],  
  'unstay': [],
  'unstay_len': [], 
#   'unfinish2': [],
#   'unfinish2_len': [],  
#   'unstay2': [],
#   'unstay2_len': [], 
#   'latest': [],
#   'latest_len': [],
#   'show': [],
#   'show_len': [],
}
for action in ACTIONS:
  m[action] = []
  m[f'{action}_len'] = []

for userid in tqdm(userids):
  m['userid'].append(userid)
  his = history[userid]
  for action in HIS_ACTIONS:
    if action in m:
      m[action].append(' '.join(map(str, his[action])))
      m[f'{action}_len'].append(len(his[action]))

100%|██████████| 20000/20000 [00:06<00:00, 3280.37it/s]


In [15]:
his_df = pd.DataFrame(m)

In [ ]:
# his_df.head()

In [ ]:
# his_df[his_df.userid == 99097].unfinish_len

In [16]:
his_df.describe(percentiles=[.25,.5,.75,.9,.95,.99, .999])

,userid,pos_len,finish_len,stay_len,neg_len,unfinish_len,unstay_len,read_comment_len,comment_len,like_len,click_avatar_len,forward_len,follow_len,favorite_len
count,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,125084.40225,25.251150,164.881750,189.393700,340.642950,52.529200,1.504500,12.81210,0.148050,9.441850,2.756400,1.398150,0.263850,0.491200
std,72559.17173,46.743077,140.299062,151.090575,234.197601,83.478597,2.397721,41.29231,1.675253,23.170245,5.728324,4.610509,1.222623,5.192078
min,8.00000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,61268.00000,5.000000,53.000000,68.000000,141.000000,11.000000,0.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,125894.50000,10.000000,124.000000,147.000000,286.000000,27.000000,1.000000,0.00000,0.000000,4.000000,1.000000,0.000000,0.000000,0.000000
75%,188072.25000,24.000000,242.000000,277.000000,509.000000,61.000000,2.000000,5.00000,0.000000,8.000000,3.000000,1.000000,0.000000,0.000000
90%,225922.40000,58.000000,371.000000,412.000000,702.000000,117.000000,4.000000,31.00000,0.000000,21.000000,8.000000,4.000000,1.000000,0.000000
95%,237992.95000,102.000000,450.000000,496.000000,789.000000,171.000000,6.000000,73.00000,1.000000,36.000000,13.000000,7.000000,1.000000,2.000000
99%,247812.08000,243.010000,587.000000,631.000000,888.010000,476.000000,11.000000,216.01000,3.000000,104.000000,27.000000,19.000000,5.000000,10.000000


In [17]:
his_df.reset_index().to_feather('../input/history.feather')